In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv("./Output/Takhsis/allocate1_sol.csv")


In [3]:
def f(s):
    res= re.findall(r'\((\d+), (\d+), (\d+), (\d+)',s)
    return tuple([int(i) for i in res[0]])


df['Unnamed: 0'] = df['Unnamed: 0'].apply(f)
df.rename(columns={'Unnamed: 0':0,'0':'sol'}, inplace=True)

In [4]:
multi=pd.MultiIndex.from_tuples(df[0])
df.set_index(multi,inplace=True)
df.drop(columns=0, inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'level_0': 'd', 'level_1': 'w', 'level_2': 'p', 'level_3':'i'}, inplace=True)
df['sol'] = df['sol'].round(0).astype(int)

df


,d,w,p,i,sol
0,0,0,0,0,0
1,0,0,0,1,0
2,0,0,0,2,0
3,0,0,0,3,0
4,0,0,0,4,0
...,...,...,...,...,...
13648,29,40,0,0,0
13649,29,40,0,1,0
13650,29,40,0,2,0
13651,29,40,1,0,0


In [5]:
med_first_group_index = [[0, 1, 2, 5, 6, 7], [0, 1, 2, 5, 6, 7], [0, 1, 2, 3, 5, 7], [2, 3, 8, 10, 11], [1, 3, 4, 8, 9, 10, 11], [3, 4, 5, 6, 8, 9, 10, 11, 13], [0, 1, 2, 4, 5, 6, 7, 8, 9, 12, 13], [0, 2, 5, 6, 7, 11, 12, 13], [
    0, 1, 2, 3, 5, 7, 12, 13], [1, 2, 3, 8, 9, 10, 11, 12], [1, 3, 4, 8, 9, 10, 11], [3, 4, 5, 6, 8, 9, 10, 13], [1, 2, 4, 5, 6, 7, 8, 12, 13], [2, 5, 6, 7, 9, 12, 13], [1, 2, 3, 5, 7, 12, 13], [1, 2, 3, 8, 9, 10, 12], [1, 3, 4, 8, 9, 10], [3, 4, 8, 9, 10]]
med_last_group_index = [[0, 2, 9, 10, 11], [0, 2, 5, 9, 10, 11], [0, 2, 3, 5, 8, 9, 10], [1, 3, 4, 5, 7, 8], [1, 3, 4, 5, 6, 7, 8, 9], [
    2, 3, 4, 6, 7, 8, 11, 12], [0, 2, 10, 11, 12], [0, 2, 5, 9, 10, 11], [0, 3, 5, 8, 9, 10], [1, 3, 4, 5, 6, 7, 8, 9], [1, 3, 4, 5, 6, 7, 8, 9], [4, 6, 7]]

downhaul_grouping_mfirst_index = [[0, 9], [0, 9], [0, 6], [0, 6], [0, 1, 10], [0, 1], [0, 1, 3], [0, 1, 3], [
    0, 1, 3], [0, 3, 4], [0, 4, 18], [0, 4, 18], [0, 2], [0, 2], [0, 2, 8], [0, 2, 5], [0, 5], [0, 5]]
backhaul_grouping_mlast_index = [[0, 2], [0, 2, 19], [0, 16, 19], [0, 1], [0, 1, 13], [
    0, 13, 14], [0, 14], [0, 15, 17], [0, 12, 17], [0, 11, 17], [0, 11], [0]]

route_descr = ['IKA', 'MHD', 'IFN', 'AWZ', 'SRY', 'AZD', 'KER', 'RAS', 'ADU',
               'TBZ', 'JWN', 'ZAH', 'BND', 'XBJ', 'OMH', 'BUZ', 'KSH', 'GBT', 'HDM', 'SYZ']
items_weight = [166, 165, 164, 163, 162, 161, 160, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134,
                133, 132, 131, 130, 129, 128, 127, 126, 115, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 1]
# items_weight = [180, 172, 160, 135, 115, 85, 1]


In [6]:
out = pd.DataFrame(columns=['iDay', 'Day', 'iSize', 'Size', 'iMed-Hotel', 'Med-Hotel-Index', 'iMec-Group', 'Mec-Group', 'iStation', 'Station'])

for d in range(df['d'].iloc[-1]+1):
    for w in range(df[df['d'] == 0]['w'].iloc[-1]+1):
        for i in range(sum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 1)]['sol'])):
            out = pd.concat([out, pd.DataFrame([[
                d,
                d+1,

                w,
                items_weight[w],

                list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 0)]['sol']) > i).index(True),
                (med_first_group_index+med_last_group_index)[d][list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 0)]['sol']) > i).index(True)],

                list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 1)]['sol']) > i).index(True),
                list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 1)]['sol']) > i).index(True)+1,
                     
                (downhaul_grouping_mfirst_index+backhaul_grouping_mlast_index)[d][list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 2)]['sol']) > i).index(True)],
                route_descr[(downhaul_grouping_mfirst_index+backhaul_grouping_mlast_index)[d][list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 2)]['sol']) > i).index(True)]],
            ]],
                columns=['iDay', 'Day', 'iSize', 'Size', 'iMed-Hotel', 'Med-Hotel-Index', 'iMec-Group', 'Mec-Group', 'iStation', 'Station'])],
                axis=0, ignore_index=True)

out


,iDay,Day,iSize,Size,iMed-Hotel,Med-Hotel-Index,iMec-Group,Mec-Group,iStation,Station
0,0,1,7,144,1,1,0,1,9,TBZ
1,0,1,7,144,2,2,0,1,9,TBZ
2,0,1,7,144,5,7,0,1,9,TBZ
3,0,1,9,142,3,5,0,1,0,IKA
4,0,1,19,132,0,0,0,1,0,IKA
...,...,...,...,...,...,...,...,...,...,...
367,28,29,33,90,0,1,2,3,11,ZAH
368,29,30,11,140,2,7,0,1,0,IKA
369,29,30,33,90,0,4,0,1,0,IKA
370,29,30,33,90,0,4,0,1,0,IKA


Later job: Handle 1 size carevans

In [11]:
list(np.cumsum(df[(df['d'] == d) & (df['w'] == w) & (df['p'] == 1)]['sol']) > i).index(True)


ValueError: True is not in list

In [ ]:
# station_items = out.groupby(['iStation', 'Size'])['iSize'].count().unstack(fill_value=0).astype(int)
# station_items.sort_index(axis=1, ascending=False, inplace=True)
# station_items = station_items.values.tolist()


In [ ]:
# items_weight=list(-np.sort(-out['Size'].unique()))
# len(items_weight)


In [7]:
# Inserting carevan sizes that are don't exist in station_items with all values equal to 0
station_items = out.groupby(['iStation', 'Size'])['iSize'].count().unstack(fill_value=0).astype(int)

to_insert_nonexist_carevan_size = {el: [0]*len(station_items) for el in pd.Series(items_weight)[~pd.Series(
    items_weight).isin(out.groupby(['iStation', 'Size'])['iSize'].count().unstack(fill_value=0).columns)].values}
station_items = pd.concat([station_items, pd.DataFrame(to_insert_nonexist_carevan_size)], axis=1)
station_items.sort_index(axis=1, ascending=False, inplace=True)
station_items = station_items.values.tolist()


In [10]:
station_items= pd.DataFrame(station_items).fillna(0).values.tolist()


In [11]:
len(items_weight)

41

In [ ]:
# out[(out['iStation'] == 1)]['Size'].unique().sort()#[['iStation','Station','Size']]


In [ ]:
# out[(out['iMed-Hotel'] == 0)][['Med-Hotel','Station', 'Size']]


In [12]:
import json
station_province_index = [[0, 1, 2, 3, 4, 5, 6, 7], [9, 10], [11, 12], [13], [14], [15], [
    16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28, 29], [30, 31]]
province_names = json.loads("[\"تهران‌\", \"البرز\", \"قزوين\", \"قم\", \"كاشان\", \"سمنان\", \"لرستان\", \"مركزي\", \"گیلان\", \"خراسان رضوي\", \"خراسان شمالي\", \"اصفهان\", \"چهارمحال بختياري\", \"اهواز\", \"مازندران\", \"یزد\", \"کرمان\", \"خالی\", \"اردبیل\", \"تبریز\", \"زنجان\", \"زاهدان\", \"هرمزگان\", \"خراسان جنوبی\", \"آذربایجان غربی\", \"بوشهر\", \"كرمانشاه\", \"ايلام\", \"گلستان\", \"همدان\", \"كردستان\", \"فارس\", \"كهگيلويه و بوير احمد\"]")
# station_province_size = [[15143, 1516, 993, 2389, 834, 1256, 1511, 1466], [9810, 1374], [8100, 521], [4194], [4660], [3728], [
#     3029], [699], [699], [4427], [1165], [1631], [932], [2097], [2796], [1165], [812, 120], [2563], [2585, 1609], [3588, 140]]
# station_province_sunni = [[0, 0, 0, 0, 0, 0, 0, 0], [745, 0], [0, 0], [0], [0], [0], [0], [0], [
#     0], [0], [0], [1219], [599], [282], [1162], [130], [354, 0], [1659], [0, 1466], [352, 0]]

# Ilam 120 is problematic - we don't have 120 carevan
# We check all kermanshah stations by out[out['Station'] == 'KSH']  and reassign ilam with available carevans near 120, which is 130
# kermanshah [812, 120] => [802, 130]
station_province_size = [[5244, 400, 360, 1305, 360, 495, 405, 495], [5440, 540], [4192, 228], [2340], [1560], [
    1820], [1300], [0], [260], [1560], [520], [1560], [520], [1040], [1040], [520], [260], [1560], [835, 465], [1247, 53]]

station_province_sunni = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [606, 70], [0, 0], [0], [0], [0], [
    0], [0], [0], [0], [0], [1371], [387], [191], [478], [95], [132], [1280], [0, 413], [273, 0]]

station_passengers = [9230, 5980, 4420, 2340, 1560, 1820, 1300, 0,
                      260, 1560, 520, 1560, 520, 1040, 1040, 520, 260, 1560, 1300, 1300]



In [ ]:
# WHy infeasible - len 55
# print(np.array(station_items).dot(items_weight))
# print(station_passengers)
# for s in range(20):
#     for p in station_province_index[s]:
#         print(station_passengers[s])



In [13]:
import gurobipy as gp
from gurobipy import GRB
opt_province_size= None
opt_province_sunni = None

def create_province_carevan_model(target_province_size, target_province_sunni, station_items, station_province_index, station_passengers, items_weight):
     opt_model = gp.Model(name="Province-Carevan Allocation")

     num_provinces = station_province_index[-1][-1]+1
     num_weight= len(items_weight)
     num_stations = len(station_passengers)

     x = {(p,w): opt_model.addVar(vtype=GRB.INTEGER, lb=0, name='X(%i_%i)' % (p,w))
          for p in range(num_provinces) for w in range(num_weight)}
     sunn = {(p, w): opt_model.addVar(vtype=GRB.INTEGER, lb=0, name='SUN(%i_%i)' % (p, w))
                    for p in range(num_provinces) for w in range(num_weight)}
     
     # Sunni is subset of x (total carevans for each province)
     opt_model.addConstrs(sunn[p,w] <= x[p,w] 
                              for p in range(num_provinces) for w in range(num_weight))
     
     # REMOVED:All station passengers should be assigned to a province
     # opt_model.addConstrs(x[p,w] * items_weight[w] == station_passengers[s]
     #                          for s in range(num_stations) for p in station_province_index[s] for w in range(num_weight))
     
     # The items weight for a station must be equal to sum of all items weights for its relevant province
     opt_model.addConstrs(gp.quicksum(x[p, w] for p in station_province_index[s]) == station_items[s][w]
                          for s in range(num_stations) for w in range(num_weight))
                         
     #Objective function include 2 statement: 1)Size Error 2)Sunni Error
     # Auxiliary variable for absolute error
     size_error_obj = {(s,p): opt_model.addVar(lb=-1000000, name='SIZE_ERR(%i_%i)' % (s,p))
                       for s in range(num_stations) for p in station_province_index[s]}
     sunni_error_obj = {(s, p): opt_model.addVar(lb=-1000000, name='SUNN_ERR(%i_%i)' % (s, p))
                        for s in range(num_stations) for p in station_province_index[s]}

     opt_model.addConstrs(size_error_obj[s, p] == 
                                        target_province_size[s][station_province_index[s].index(p)]
                                        - gp.quicksum(x[p, w]*items_weight[w] for w in range(num_weight))
               for s in range(num_stations) for p in station_province_index[s])
                         
     opt_model.addConstrs(sunni_error_obj[s, p] ==
                                        target_province_sunni[s][station_province_index[s].index(p)]
                                        - gp.quicksum(sunn[p, w]*items_weight[w] for w in range(num_weight))
               for s in range(num_stations) for p in station_province_index[s])

     size_abs_error_obj = {(s, p): opt_model.addVar(name='SIZE_ABS_ERR(%i_%i)' % (s, p))
                           for s in range(num_stations) for p in station_province_index[s]}
     sunni_abs_error_obj = {(s, p): opt_model.addVar(name='SUNN_ABS_ERR(%i_%i)' % (s, p))
                            for s in range(num_stations) for p in station_province_index[s]}

     opt_model.addConstrs(size_abs_error_obj[s, p] == gp.abs_(size_error_obj[s,p])
                          for s in range(num_stations) for p in station_province_index[s])
     opt_model.addConstrs(sunni_abs_error_obj[s, p] == gp.abs_(sunni_error_obj[s,p])
                          for s in range(num_stations) for p in station_province_index[s])

     obj1 = gp.quicksum(size_abs_error_obj[s,p]
                        for s in range(num_stations) for p in station_province_index[s])
     obj2 = gp.quicksum(sunni_abs_error_obj[s,p]
                        for s in range(num_stations) for p in station_province_index[s])
     opt_model.ModelSense = GRB.MINIMIZE
     opt_model.setObjective(obj1 + obj2)
     
     # opt_model.write("out-param.lp")
     # opt_model.write("out-param.mps")

     # Model Parameters
     opt_model.Params.timelimit = 10000
     opt_model.Params.integralityfocus = 1
     # opt_model.Params.mipfocus = 1

     def getSol(model, where):
          if where == GRB.Callback.MIPSOL:
               global opt_province_size
               global opt_province_sunni

               opt_province_size = model.cbGetSolution(x)
               opt_province_sunni = model.cbGetSolution(sunn)
               
     opt_model._vars = opt_model.getVars()
     opt_model.optimize(getSol)


In [15]:
prov_car_model_stat= create_province_carevan_model(station_province_size, station_province_sunni, station_items,
                              station_province_index, station_passengers, items_weight)


Set parameter TimeLimit to value 10000
Set parameter IntegralityFocus to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2194 rows, 2748 columns and 6499 nonzeros
Model fingerprint: 0x3030644b
Model has 62 general constraints
Variable types: 124 continuous, 2624 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+06, 1e+06]
  RHS range        [1e+00, 5e+03]
Presolve removed 2146 rows and 2540 columns
Presolve time: 0.44s
Presolved: 48 rows, 208 columns, 424 nonzeros
Variable types: 0 continuous, 208 integer (64 binary)
Found heuristic solution: objective 15855.000000
Found heuristic solution: objective 15525.000000

Root relaxation: objective 3.070000e+02, 59 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

In [16]:
prov_car_model_stat


In [ ]:
# Sort carevans by Medina hotels index and mec group. The sooner province will be assigned to sooner medina hotels(more expensive)
# The iMec groups sort is descending, meaning sooner province will go to longer (1 day) carevans
# Therefore, first provinces will have more expensive package in terms of medina hotels and longer total durations. 
#   This will impose a more expensive mecca hotel later
# out[(out['iStation'] == 0) & (out['iSize'] == 0)].sort_values(['iMed-Hotel', 'iMec-Group'], ascending=[True, False])#.index


In [17]:
df_province_size = pd.DataFrame.from_dict(opt_province_size, orient='index')
multi = pd.MultiIndex.from_tuples(df_province_size.index)
df_province_size.set_index(multi, inplace=True)
df_province_size.reset_index(inplace=True)
df_province_size.rename(columns={'level_0': 'pr', 'level_1': 'w', 0: 'sol'}, inplace=True)
df_province_size['sol'] = df_province_size['sol'].round(0).astype(int)
# df_province_size


In [18]:
def spreadProvinceID(a):
    res=[]
    for i in a[a>0].index:
        res+= a[i]*[i]
    return res

def spreadSunniVal(a):
    res = []
    for i in a[a > 0].index:
        res += a[i]*[1]
    return res


In [56]:
np.cumsum(df[(df['d'] == 43)]['sol']) > 0


31436    False
31437    False
31438    False
31439    False
31440    False
         ...  
31953     True
31954     True
31955     True
31956     True
31957     True
Name: sol, Length: 522, dtype: bool

In [19]:
out[(out['iStation'] == 5) & (out['iSize'] == 25)].sort_values(
    ['Med-Hotel-Price','iMec-Group'], ascending=[ False,False])


KeyError: 'Med-Hotel-Price'

In [42]:
df_province_size[df_province_size['pr'].isin(station_province_index[7])].groupby(
    ['pr', 'w']).sum().unstack().apply(spreadProvinceID)[0]
    



KeyError: 0

In [55]:
df_province_size[df_province_size['pr'].isin(station_province_index[7])].groupby(
['pr', 'w']).sum().sum()[0]==0


True

In [57]:
# Sort carevans by Medina hotels index and mec group. The sooner province will be assigned to sooner medina hotels(more expensive)
# The iMec groups sort is descending, meaning sooner province will go to longer (1 day) carevans
# Therefore, first provinces will have more expensive package in terms of medina hotels and longer total durations.
#   This will impose a more expensive mecca hotel later
out['iProvince'] = None

for s in range(len(station_passengers)):
    for w in range(len(items_weight)):
        if df_province_size[df_province_size['pr'].isin(station_province_index[s])].groupby(
            ['pr', 'w']).sum().sum()[0] == 0:
            continue
        out.loc[
            out[(out['iStation'] == s) & (out['iSize'] == w)].sort_values(['iMed-Hotel', 'iMec-Group'], ascending=[True, False]).index,
            'iProvince'] = \
            df_province_size[df_province_size['pr'].isin(station_province_index[s])].groupby(['pr', 'w']).sum().unstack().apply(spreadProvinceID)[w]
    



In [58]:
out['Province'] = out['iProvince'].apply(lambda x: province_names[x])


In [59]:
df_province_sunni = pd.DataFrame.from_dict(opt_province_sunni, orient='index')
multi = pd.MultiIndex.from_tuples(df_province_sunni.index)
df_province_sunni.set_index(multi, inplace=True)
df_province_sunni.reset_index(inplace=True)
df_province_sunni.rename(columns={'level_0': 'pr', 'level_1': 'w', 0: 'sol'}, inplace=True)
df_province_sunni['sol'] = df_province_sunni['sol'].round(0).astype(int)
df_province_sunni


,pr,w,sol
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
1307,31,36,0
1308,31,37,0
1309,31,38,0
1310,31,39,0


In [60]:
# Sort carevans by Medina hotels index and mec group. The sooner carevan will be assigned to later medina hotels(less expensive)
# The iMec groups sort is descending, meaning sooner province will go to longer (1 day) carevans
out['Sunni'] = 0

for s in range(len(station_passengers)):
    for w in range(len(items_weight)):
        if df_province_sunni[df_province_sunni['pr'].isin(station_province_index[s])].groupby(['pr', 'w']).sum()['sol'].sum() > 0:
            sunni_value = df_province_sunni[df_province_sunni['pr'].isin(station_province_index[s])].groupby(
                ['pr', 'w']).sum().unstack().apply(spreadSunniVal)[w]
                
            out.loc[
                out[(out['iStation'] == s) & (out['iSize'] == w)].sort_values(['iMed-Hotel', 'iMec-Group'], ascending=[True, False]).index[:len(sunni_value)],
                'Sunni'] = sunni_value

out['Sunni']=out['Sunni'].astype(int)



Now, building mecca hotel takhsis.
Result: Which carevan goes to which hotel in mecca

In [21]:
med_hotel_price=[1,0.6,0.7,0.7,0.7,0.7,0.6,0.5,0.6,0.5,0.5,0.6,0.6,0.6]
# med_hotel_name = ['NEW','مراج طيبه', 'ميراج السلام', 'المدينه موفنبيک', 'انوار الزهراء', 'المحمديه الزهرا', 'فندق روابي الزهراء', 'راما المدينه', 'الجاد المدينه',
#                   'قصر الاندلس', 'فال امجاد الغراء', 'حياه العالمي', 'ازکي المدينه', 'منازل المهاجرين', 'رياض المهاجرين', 'المدينه منازلي', 'باين وود', 'القبله منازلي', 'سبال']
med_hotel_name = json.loads(
    "[\"H1\", \"H2\", \"H3\", \"H4\", \"H5\", \"H6\", \"H7\", \"H8\", \"H9\", \"H10\", \"H11\", \"H12\", \"H13\", \"H14\", \"H15\", \"H16\", \"H17\", \"H18\", \"H19\"]")
# In future, it must be sorted, with priciest at first and cheapest at last
mec_hotel_price = [0.7, 0.9, 0.7, 0.7, 0.6, 0.6, 0.6, 0.7, 0.7, 0.5,
                   0.5, 0.8, 0.9, 0.9, 0.7, 0.8, 0.8, 0.9, 0.7, 0.6, 0.8, 0.8, 0.8, 0.7]
mec_hotel_name = json.loads("[\"السدره الاحسان \",\"ميدان \",\"رفاف الحياة \",\"برج الرائد \",\"منار التوحيد  \",\"جوار البيت 1 \",\"جوار البيت 2 \",\"الزائر \",\"سدره البيت 4 \",\"جوهره الصبغه A \",\"جوهره الصبغه B \",\"رفاف المشاعر \",\"دانه الماسيه \",\" ويولت 1 \",\"لابا دارنا \",\"جدوي صدقي برج A \",\"جدوي صدقي برج B \",\"العليان الماسي \",\"نخله الميره 8 \",\"الصاعد بلازا \",\"ويولت 3 \",\"منار التوحيد2 \",\"2ج نامعلوم\",\"2د نامعلوم\"]")
# mec_hotel_capacity = [637, 1478, 1158, 925, 1109, 1499, 833, 1032, 1298, 385, 385, 742, 240, 986, 1413, 2723, 1562, 1815, 4544, 1321, 557, 3566, 2048, 2432, 2799, 813, 618, 619, 2596, 3365, 916, 1431,
#                      2319, 1021, 558, 2287, 820, 1559, 1589, 1622, 459, 944, 555, 2710, 622, 737, 804, 1070, 1642, 652, 917, 652, 555, 1504, 1063, 1731, 464, 1410, 947, 371, 1176, 909, 987, 874, 454, 599, 1143, 1006, 1745, ]
# Real Total minus avamel up, this will be infeasible - Real Total below is feasible
# We have to remove avamel from hotels AND from station_passengers and total_passengers
mec_hotel_capacity = [805, 2648, 564, 962, 621, 383, 383, 1985, 563, 679, 242,
                      627, 2087, 2859, 1182, 1675, 1675, 2766, 563, 1347, 1460, 1507, 4925, 4925]
mec_hotel_just_sunni = [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

province_preference = [[23, 19, 17], [0, 18, 23], [28, 23, 25, 19], [15, 14, 20, 22], [8, 9, 3, 5, 22], [
    30, 24, 21, 12], [16, 2, 17], [0, 10, 4, 3, 7], [13, 27, 9, 5, 8], [6, 26, 25, 29, 23]]


In [22]:
#Making sunni index hotel and carevan
sunni_hotel_index = pd.Series(mec_hotel_just_sunni)
sunni_hotel_index = sunni_hotel_index[sunni_hotel_index==1].index.tolist()

sunni_carevan_index= out[out['Sunni']==1].index.tolist()

#Making shia idnex hotel and carevan
shia_hotel_index = pd.Series(mec_hotel_just_sunni)
shia_hotel_index = shia_hotel_index[shia_hotel_index == 0].index.tolist()

shia_carevan_index = out[out['Sunni'] == 0].index.tolist()


In [64]:
import copy
def build_mec_groups(group1, group2):
    g1 = copy.deepcopy(group1)
    g2 = copy.deepcopy(group2)
    # print("G1:")
    # print(g1)
    # print("G2:")
    # print(g2)
    # Grouping
    new_g = [[] for i in range(len(g1))]
    d1 = 0
    for i in range(len(g2)):
        bin2 = g2[i]

        for j in range(d1, len(g1)):
            bin1 = g1[j]
            if bin1 <= bin2:
                new_g[j].append(bin1)
                bin2 -= bin1
                d1 = j+1
                continue
            else:
                new_g[j].append(bin2)
                g1[j] -= bin2
                break

    # print('Main Groups')
    # for i in range(len(new_g)):
    #     print(new_g[i])
    main_groups = copy.deepcopy(new_g)
    # print("********")

    # for grups smaller than 90-3=87, join to other groups
    for i in range(len(new_g)):
        for j in range(len(new_g[i])):
            # print(i, j)
            if new_g[i][j] < 87:
                if j < len(new_g[i]) - 1:
                    # join with next
                    new_g[i][j+1] += new_g[i][j]
                    new_g[i].pop(j)
                    break
                else:
                    # join with previous
                    new_g[i][j-1] += new_g[i][j]
                    new_g[i].pop(j)
                    break

    # print('Final groups binding small groups to larger ones')
    # for i in range(len(new_g)):
    #     print(new_g[i])

    # print("Build_MEC Output")
    # print("Main Groups")
    # print(main_groups)
    # print("New G")
    # print(new_g)
    return main_groups, new_g


In [73]:
down = [456, 1388, 1854, 1622, 1630, 1630, 1863, 1863, 1863,
        1863, 1863, 1863, 1863, 1864, 2330, 2330, 2330, 2330, 2330]
back = [1864, 1864, 1864, 1864, 1864, 1864, 1864, 2097, 1864,
        2089, 2087, 1854, 2087, 2087, 2097, 2097, 2097, 1631]
a, _ = build_mec_groups(back,down)

In [74]:
a

[[456, 1388, 20],
 [1834, 30],
 [1592, 272],
 [1358, 506],
 [1124, 740],
 [1123, 741],
 [1122, 742],
 [1121, 976],
 [887, 977],
 [886, 1203],
 [660, 1427],
 [436, 1418],
 [446, 1641],
 [689, 1398],
 [932, 1165],
 [1165, 932],
 [1398, 699],
 [1631]]

In [88]:
# Creating new column: Med-First - MF_Entry/ML_Exit_Day
# Med-First
out['Med-First'] = 0
out.loc[out[out['Day'] <= 26].index, 'Med-First'] = 1
# MF_Entry/ML_Exit_Day
out.loc[out[out['Day'] <= 26].index, 'Day-Code'] = out[out['Day'] <= 26]['Day']
out.loc[out[out['Day'] > 26].index, 'Day-Code'] = out[out['Day'] > 26]['Day'] - 26
out


,iDay,Day,iSize,Size,iMed-Hotel,Med-Hotel-Index,iMec-Group,Mec-Group,iStation,Station,iProvince,Province,Sunni,Med-Hotel-Price,Med-Hotel-Name,Med-First,Day-Code
0,0,1,0,192,4,8,0,1,0,IKA,3,قم,0,0.8,H9,1,1
1,0,1,1,191,6,10,0,1,9,TBZ,18,تبریز,0,0.6,H11,1,1
2,0,1,20,172,4,8,0,1,9,TBZ,18,تبریز,0,0.8,H9,1,1
3,0,1,24,168,5,9,0,1,9,TBZ,18,تبریز,0,0.5,H10,1,1
4,0,1,24,168,5,9,0,1,9,TBZ,18,تبریز,0,0.5,H10,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,43,44,23,169,1,2,0,1,17,GBT,27,گلستان,0,1.0,H3,0,18
597,43,44,25,144,0,0,0,1,11,ZAH,20,زاهدان,0,0.5,H1,0,18
598,43,44,25,144,0,0,0,1,14,OMH,23,آذربایجان غربی,1,0.5,H1,0,18
599,43,44,27,142,3,9,0,1,11,ZAH,20,زاهدان,1,0.5,H10,0,18


In [103]:
out[(out['Sunni'] == 1)]


,iDay,Day,iSize,Size,iMed-Hotel,Med-Hotel-Index,iMec-Group,Mec-Group,iStation,Station,iProvince,Province,Sunni,Med-Hotel-Price,Med-Hotel-Name,Med-First,Day-Code
87,6,7,12,180,3,6,1,2,12,BND,21,هرمزگان,1,0.5,H7,1,7
93,6,7,26,143,5,9,1,2,12,BND,21,هرمزگان,1,0.5,H10,1,7
94,6,7,26,143,6,10,1,2,12,BND,21,هرمزگان,1,0.6,H11,1,7
99,7,8,0,192,8,18,0,1,16,KSH,25,كرمانشاه,1,0.6,H19,1,8
110,7,8,36,133,2,4,0,1,12,BND,21,هرمزگان,1,0.5,H5,1,8
116,8,9,14,178,1,5,1,2,16,KSH,25,كرمانشاه,1,0.6,H6,1,9
122,8,9,27,142,0,1,0,1,15,BUZ,24,بوشهر,1,0.6,H2,1,9
161,11,12,25,144,0,0,0,1,1,MHD,8,خراسان رضوي,1,0.5,H1,1,12
333,23,24,27,142,2,3,1,2,13,XBJ,22,خراسان جنوبی,1,0.6,H4,1,24
345,24,25,29,140,0,0,0,1,13,XBJ,22,خراسان جنوبی,1,0.5,H1,1,25


In [143]:
out[out['Sunni'] == 1]['Med-Hotel-Price'].tolist()

[0.5,
 0.5,
 0.6,
 0.6,
 0.5,
 0.6,
 0.6,
 0.5,
 0.6,
 0.5,
 0.6,
 1.0,
 0.5,
 1.0,
 0.6,
 0.6,
 0.5,
 0.6,
 0.6,
 0.8,
 0.6,
 0.8,
 0.8,
 0.8,
 0.6,
 0.6,
 0.6,
 0.5,
 0.5,
 0.6,
 0.5,
 0.5,
 0.6,
 0.6,
 0.5,
 0.6,
 0.6,
 0.5,
 0.6,
 0.6,
 1.0,
 1.0,
 0.5,
 0.5,
 1.0,
 0.6,
 0.6,
 0.5,
 1.0,
 0.5,
 0.5,
 0.5]

In [108]:
out['Med-Hotel-Index'].apply(lambda x: med_hotel_price[x]).iloc[sunni_carevan_index].tolist()

0.5

In [23]:
opt_mec_takhsis_shia = None
opt_mec_takhsis_sunni= None


def create_mecca_takhsis_model(med_hotel_price, mec_hotel_price, mec_hotel_capacity, mec_hotel_just_sunni, province_preference, sect_hotel_index, sect_carevan_index, sunni, hotel_class_toler ,car_df):
     opt_model = gp.Model(name="Mecca Hotel Allocation")

     num_hotels = len(sect_hotel_index)#len(mec_hotel_price)
     num_carevans = len(sect_carevan_index)  # len(car_df)
     

     x = {(h, c): opt_model.addVar(vtype=GRB.BINARY, name='X(%i_%i)' % (h,c))
          for h in range(num_hotels) for c in range(num_carevans)}

     # y means if hotel i is empty or has passengers (semi-full) - just required for objective
     y = {(h): opt_model.addVar(vtype=GRB.BINARY, name='Y(%i)' % (h))
          for h in range(num_hotels)}
     opt_model.addConstrs(y[h] == gp.any_(x[h,c] for c in range(num_carevans))
                         for h in range(num_hotels))

     # Constraints
     # Each carevan should be assigned only to 1 hotel
     opt_model.addConstrs(gp.quicksum(x[h,c] for h in range(num_hotels)) == 1
                         for c in range(num_carevans))

     # Hotel Capacity Constraint
     opt_model.addConstrs(gp.quicksum(x[h, c]*car_df['Size'][sect_carevan_index[c]] for c in range(num_carevans)) <= mec_hotel_capacity[sect_hotel_index[h]]
                              for h in range(num_hotels))
     
     # Mec price should be >= of med price
     car_med_price = car_df['Med-Hotel-Index'].apply(lambda x: med_hotel_price[x]).iloc[sect_carevan_index].tolist()
     opt_model.addConstrs(gp.quicksum(x[h, c] * mec_hotel_price[sect_hotel_index[h]] for h in range(num_hotels)) >= car_med_price[c] - hotel_class_toler
                         for c in range(num_carevans))

     # Limit just-sunni hotels
     # just_sunni_hotels= pd.Series(mec_hotel_just_sunni)
     # just_sunni_hotels = just_sunni_hotels[just_sunni_hotels==1].index.tolist()

     # non_sunni_carevans = car_df[car_df['Sunni'] == 0].index.tolist()
     
     # opt_model.addConstrs(gp.quicksum(x[h,c] for c in non_sunni_carevans) == 0
     #                     for h in just_sunni_hotels)

     # Setting objective function
     # obj1 is for cost reduction - making some hotels (more expensive) empty as much as possible
     obj1= gp.quicksum(y[h]*mec_hotel_price[sect_hotel_index[h]] for h in range(num_hotels))
     
     #obj2 is for preference of some provinces be together
     # z[h,pref] means hotel h contains preferenc pref if == 1, otherwise 0
     num_prov_pref= len(province_preference)

     z = {(h,pref): opt_model.addVar(vtype=GRB.BINARY, name='Z(%i_%i)' % (h, pref))
          for h in range(num_hotels) for pref in range(num_prov_pref)}
     
     opt_model.addConstrs(z[h, pref] == gp.any_(x[h, c]
                                             for c in [sect_carevan_index.index(i) for i in
                                                  car_df[(car_df['iProvince'].isin(province_preference[pref])) & (car_df['Sunni'] == sunni)].index]
                                             )
                         for h in range(num_hotels) for pref in range(num_prov_pref))

     # zhotel equals number of preference existing in hotel h - 1 . It should be close to only 1
     zhotel = {(h): opt_model.addVar(vtype=GRB.INTEGER, name='ZH(%i)' % (h))
          for h in range(num_hotels)}

     opt_model.addConstrs(zhotel[h] == gp.quicksum(z[h,pref] for pref in range(num_prov_pref)) - 1
                         for h in range(num_hotels))
     
     # zhotel_abs is the absolute value of |zhotel - 1|
     zhotel_abs = {(h): opt_model.addVar(vtype=GRB.INTEGER, name='ZH_ABS(%i)' % (h))
               for h in range(num_hotels)}
     
     opt_model.addConstrs(zhotel_abs[h] == gp.abs_(zhotel[h])
                         for h in range(num_hotels))
     
     obj2= gp.quicksum(zhotel_abs[h] for h in range(num_hotels))
     
     # Setting obj and confuguration
     opt_model.ModelSense = GRB.MINIMIZE
     opt_model.setObjective(3*obj1 + obj2)

     # opt_model.write("out-param.lp")
     opt_model.write("mec_alloc_out_param.mps")

     # Model Parameters
     opt_model.Params.timelimit = 10000
     opt_model.Params.integralityfocus = 1
     opt_model.Params.mipfocus = 1
     # opt_model.Params.heuristics = 0.2

     def getSol(model, where):
          global opt_mec_takhsis_sunni
          global opt_mec_takhsis_shia

          if where == GRB.Callback.MIPSOL:
               if sunni == 1:
                    opt_mec_takhsis_sunni = model.cbGetSolution(x)
               else:
                   opt_mec_takhsis_shia = model.cbGetSolution(x)

     opt_model._vars = opt_model.getVars()
     opt_model.optimize(getSol)


In [31]:
#Run Shia
create_mecca_takhsis_model(med_hotel_price, mec_hotel_price, mec_hotel_capacity, mec_hotel_just_sunni,
                           province_preference, shia_hotel_index, shia_carevan_index, 0, 0, out)


Set parameter TimeLimit to value 10000
Set parameter IntegralityFocus to value 1
Set parameter MIPFocus to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1278 rows, 34987 columns and 103309 nonzeros
Model fingerprint: 0x74d0454f
Model has 708 general constraints
Variable types: 0 continuous, 34987 integer (34869 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 5e+03]
Presolve added 55725 rows and 0 columns
Presolve removed 0 rows and 11361 columns
Presolve time: 1.87s
Presolved: 57003 rows, 23626 columns, 192681 nonzeros
Variable types: 0 continuous, 23626 integer (23567 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    99

In [32]:
# Reading result of neos
opt_mec_takhsis_shia= {}
with open('/Users/ali/Downloads/model-5.sol') as f:
    content = f.read()
    entry1 = re.findall(r'X\((\d+)_(\d+)\) (\d+)', content)
    for x in entry1:
        opt_mec_takhsis_shia.update({(int(x[0]), int(x[1])) : int(x[2])})


In [33]:
df_mec_takhsis_shia = pd.DataFrame.from_dict(opt_mec_takhsis_shia, orient='index')
multi = pd.MultiIndex.from_tuples(df_mec_takhsis_shia.index)
df_mec_takhsis_shia.set_index(multi, inplace=True)
df_mec_takhsis_shia.reset_index(inplace=True)
df_mec_takhsis_shia.rename(columns={'level_0': 'h', 'level_1': 'car', 0: 'sol'}, inplace=True)
df_mec_takhsis_shia['sol'] = df_mec_takhsis_shia['sol'].round(0).astype(int)
df_mec_takhsis_shia


,h,car,sol
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
34215,58,575,0
34216,58,576,0
34217,58,577,0
34218,58,578,0


In [34]:
res= df_mec_takhsis_shia[df_mec_takhsis_shia['sol']==1]
out['Mec-Hotel-Index']=None

out.loc[[shia_carevan_index[c] for c in res['car'].values],'Mec-Hotel-Index'] = [shia_hotel_index[h] for h in res['h'].values]


In [35]:
# create_mecca_takhsis_model(med_hotel_price, mec_hotel_price, mec_hotel_capacity,
#                            mec_hotel_just_sunni, province_preference, out)

#Run Sunni
create_mecca_takhsis_model(med_hotel_price, mec_hotel_price, mec_hotel_capacity, mec_hotel_just_sunni,
 province_preference, sunni_hotel_index, sunni_carevan_index, 1, 0.2, out)

Set parameter TimeLimit to value 10000
Set parameter IntegralityFocus to value 1
Set parameter MIPFocus to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 130 rows, 680 columns and 1760 nonzeros
Model fingerprint: 0xc9a03418
Model has 120 general constraints
Variable types: 0 continuous, 680 integer (660 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+02]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 2e+03]
Presolve added 814 rows and 0 columns
Presolve removed 0 rows and 116 columns
Presolve time: 0.04s
Presolved: 944 rows, 564 columns, 3510 nonzeros
Variable types: 0 continuous, 564 integer (564 binary)
Found heuristic solution: objective 61.3000000

Root relaxation: objective 2.230000e+01, 701 iterations, 0.04 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     W

In [36]:
df_mec_takhsis_sunni = pd.DataFrame.from_dict(opt_mec_takhsis_sunni, orient='index')
multi = pd.MultiIndex.from_tuples(df_mec_takhsis_sunni.index)
df_mec_takhsis_sunni.set_index(multi, inplace=True)
df_mec_takhsis_sunni.reset_index(inplace=True)
df_mec_takhsis_sunni.rename(columns={'level_0': 'h', 'level_1': 'car', 0: 'sol'}, inplace=True)
df_mec_takhsis_sunni['sol'] = df_mec_takhsis_sunni['sol'].round(0).astype(int)
df_mec_takhsis_sunni


,h,car,sol
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
545,9,50,0
546,9,51,0
547,9,52,0
548,9,53,0


In [37]:
res = df_mec_takhsis_sunni[df_mec_takhsis_sunni['sol'] == 1]

out.loc[[sunni_carevan_index[c] for c in res['car'].values],'Mec-Hotel-Index'] = [sunni_hotel_index[h] for h in res['h'].values]


In [28]:
#Adding Hotel price columns for med and mec
out['Med-Hotel-Price']= out['Med-Hotel-Index'].apply(lambda x: med_hotel_price[x])
out['Med-Hotel-Name'] = out['Med-Hotel-Index'].apply(lambda x: med_hotel_name[x])

out['Mec-Hotel-Price'] = out['Mec-Hotel-Index'].apply(lambda x: mec_hotel_price[x])
out['Mec-Hotel-Name'] = out['Mec-Hotel-Index'].apply(lambda x: mec_hotel_name[x])


KeyError: 'Mec-Hotel-Index'

In [ ]:
out.to_excel('Carevan_list.xlsx', 'کاروان')


In [54]:
out[['Day', 'Mec-Group', 'Size', 'Station', 'Province', 'Sunni',
    'Med-Hotel-Index', 'Med-Hotel-Name', 'Med-Hotel-Price', 'Mec-Hotel-Index', 'Mec-Hotel-Name', 'Mec-Hotel-Price']].to_json(orient='split',force_ascii=False, index=False,)


'{"columns":["Day","Mec-Group","Size","Station","Province","Sunni","Med-Hotel-Index","Med-Hotel-Name","Med-Hotel-Price","Mec-Hotel-Index","Mec-Hotel-Name","Mec-Hotel-Price"],"data":[[1,1,168,"KER","کرمان",0,9,"H10",0.5,58,"الياسمين",0.5],[1,1,168,"KER","کرمان",0,9,"H10",0.5,58,"الياسمين",0.5],[1,1,144,"IKA","تهران\u200c",0,3,"H4",0.6,35,"اسکاي ان",0.6],[1,1,144,"KER","کرمان",0,3,"H4",0.6,21,"الروضه کنکورد",0.8],[1,1,144,"TBZ","تبریز",0,4,"H5",0.5,27,"ديار المشاعر",0.5],[1,1,144,"TBZ","تبریز",0,12,"H13",0.6,30,"نسمات مكه 2",0.6],[1,1,143,"IKA","تهران\u200c",0,18,"H19",0.6,35,"اسکاي ان",0.6],[1,1,141,"IKA","تهران\u200c",0,0,"H1",0.5,5,"راحه بكه",0.5],[1,1,141,"IKA","تهران\u200c",0,1,"H2",0.6,24,"برج الصقريه",1.0],[1,1,135,"KER","کرمان",0,17,"H18",0.6,8,"مكارم ام القري",1.0],[1,1,135,"TBZ","تبریز",0,17,"H18",0.6,30,"نسمات مكه 2",0.6],[1,1,130,"IKA","كاشان",0,18,"H19",0.6,44,"العليان بلازا",0.6],[1,1,96,"TBZ","تبریز",0,4,"H5",0.5,37,"ديارالجعفريه",0.5],[1,1,96,"TBZ","تبریز",0,12,"H13",0.6,